<a href="https://colab.research.google.com/github/mttcnnff/MobileNetV2/blob/master/src/mobilenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/mttcnnff/MobileNetV2.git

Cloning into 'MobileNetV2'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 47 (delta 7), reused 35 (delta 3), pack-reused 0
Unpacking objects: 100% (47/47), done.


In [0]:
# setup path
import sys
sys.path.append('/content/MobileNetV2/src')

In [0]:
import tensorflow as tf
import tensorflow_datasets as tfds
from mobilenet import mobilenet_v2
import matplotlib.pyplot as plt

LABELS = [
    'apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 
    'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 
    'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 
    'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur', 
    'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 
    'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion',
    'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse',
    'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear',
    'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine',
    'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose',
    'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake',
    'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table',
    'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout',
    'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman',
    'worm'
]

In [0]:
def load_data():
    cifar100 = tf.keras.datasets.cifar100
    return cifar100.load_data()

In [0]:
def display_x(x):
  plt.figure()
  plt.imshow(x)

In [0]:
def display_y(y):
  print(LABELS[y[0]])

In [0]:
def display_item(x, y, idx):
  display_x(x[idx])
  display_y(y[idx])

In [0]:

def batch_features_labels(features, labels, batch_size):
    """
    Split features and labels into batches
    """
    for start in range(0, len(features), batch_size):
        end = min(start + batch_size, len(features))
        yield features[start:end], labels[start:end]

In [0]:
def load_preprocess_training_batch(batch_id, batch_size, x_train_data, y_train_data):
    """
    Load the Preprocessed Training data and return them in batches of <batch_size> or less
    """
    
    start_idx = batch_id * batch_size
    
    
    
    end_idx = min(start_idx + batch_size, len())
    
    
    
    # Return the training data in batches of size <batch_size> or less
    return batch_features_labels(x_train_data, y_train_data, batch_size)

In [0]:
def main():
    tf.reset_default_graph()
    (x_train, y_train), (x_test, y_test) = load_data()
        
    x_train_tensor = tf.convert_to_tensor(x_train, dtype=tf.float32)
    y_train_tensor = tf.squeeze(tf.convert_to_tensor(y_train, dtype=tf.int32))
    
    print(x_train_tensor.get_shape())
    print(y_train_tensor.get_shape())
    
#     with tf.contrib.slim.arg_scope(mobilenet_v2.training_scope()):
#       logits, endpoints = mobilenet_v2.mobilenet(x_train_tensor, num_classes=100)
      
#       one_hot_y = tf.squeeze(tf.one_hot(y_train_tensor, 100))
      
#       # Define a loss function
#       loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = one_hot_y, 
#                                                                     logits = logits))
      
#       # Define an optimizer 
#       train_op = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss)
      
#       # Convert logits to label indexes
#       correct_pred = tf.argmax(logits, 1)

#       # Define an accuracy metric
#       accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
      
#       tf.set_random_seed(1234)
#       sess = tf.Session()

#       sess.run(tf.global_variables_initializer())

#       for i in range(100):
#         print('EPOCH', i)
#         _, accuracy_val = sess.run([train_op, accuracy])
#         if i % 10 == 0:
#           print("Loss: ", loss)
#         print('DONE WITH EPOCH')
      

In [14]:

main()

(50000, 32, 32, 3)
(50000,)


In [34]:
# Load a given dataset by name, along with the DatasetInfo
data, info = tfds.load("cifar100", with_info=True)
train_data, test_data = data['train'], data['test']
assert isinstance(train_data, tf.data.Dataset)
assert info.features['label'].num_classes == 100
assert info.splits['train'].num_examples == 50000

tf.reset_default_graph()

# Initialize placeholders 
x = tf.placeholder(dtype = tf.float32, shape = [None, 32, 32, 3])
y = tf.placeholder(dtype = tf.int32, shape = [None])

# Fully connected layer 
with tf.contrib.slim.arg_scope(mobilenet_v2.training_scope()):
  logits, endpoints = mobilenet_v2.mobilenet(x, num_classes=100)

# Define a loss function
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, 
                                                                    logits = logits))
# Define an optimizer 
train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

# Convert logits to label indexes
correct_pred = tf.argmax(logits, 1)

# Define an accuracy metric
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


with tf.Session() as sess:
    # Initializing the variables
    sess.run(tf.global_variables_initializer())
    
    # Training cycle
    for epoch in range(10):
        # Loop over all batches
        n_batches = 50
        
#         for batch_i in range(1, n_batches + 1):
#             for batch_features, batch_labels in load_preprocess_training_batch(batch_i, batch_size):
#                 train_neural_network(sess, optimizer, keep_probability, batch_features, batch_labels)
                
#             print('Epoch {:>2}, CIFAR-10 Batch {}:  '.format(epoch + 1, batch_i), end='')
#             print_stats(sess, batch_features, batch_labels, cost, accuracy)

KeyboardInterrupt: ignored

In [48]:




# batched = train_data.batch(1000)

# iter = batched.make_one_shot_iterator()
# x = iter.get_next()

# type(x)


 # Load a given dataset by name, along with the DatasetInfo
data, info = tfds.load("cifar100", with_info=True)
print(info)
train_data, test_data = data['train'], data['test']
assert isinstance(train_data, tf.data.Dataset)
assert info.features['label'].num_classes == 100
assert info.splits['train'].num_examples == 50000

dataset = train_data
first_100 = dataset.range(100)
batched_dataset = dataset.batch(4)
iterator = batched_dataset.make_one_shot_iterator()
next_element = iterator.get_next()
  
with tf.Session() as sess:
  print(sess.run(next_element)['label'])  # ==> ([0, 1, 2,   3],   [ 0, -1,  -2,  -3])
  

tfds.core.DatasetInfo(
    name='cifar100',
    version=1.3.1,
    description='This dataset is just like the CIFAR-10, except it has 100 classes containing 600 images each. There are 500 training images and 100 testing images per class. The 100 classes in the CIFAR-100 are grouped into 20 superclasses. Each image comes with a "fine" label (the class to which it belongs) and a "coarse" label (the superclass to which it belongs).',
    urls=['https://www.cs.toronto.edu/~kriz/cifar.html'],
    features=FeaturesDict({
        'coarse_label': ClassLabel(shape=(), dtype=tf.int64, num_classes=20),
        'image': Image(shape=(32, 32, 3), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=100)
    },
    total_num_examples=60000,
    splits={
        'test': <tfds.core.SplitInfo num_examples=10000>,
        'train': <tfds.core.SplitInfo num_examples=50000>
    },
    supervised_keys=('image', 'label'),
    citation='"""
        @TECHREPORT{Krizhevsky09learning

In [90]:
tf.reset_default_graph()

# Load a given dataset by name, along with the DatasetInfo
data, info = tfds.load("cifar100", with_info=True)
train_data, test_data = data['train'], data['test']
assert isinstance(train_data, tf.data.Dataset)
assert info.features['label'].num_classes == 100
assert info.splits['train'].num_examples == 50000


NUM_EXAMPLES = 100
BATCH_SIZE = 23
NUM_BATCHES = NUM_EXAMPLES//BATCH_SIZE + 1
print(NUM_BATCHES)
dataset = train_data.take(NUM_EXAMPLES)


# Initialize placeholders 
x = tf.placeholder(dtype = tf.uint8, shape = [None, 32, 32, 3])
y = tf.placeholder(dtype = tf.int64, shape = [None])

# Fully connected layer 
with tf.contrib.slim.arg_scope(mobilenet_v2.training_scope()):
  logits, endpoints = mobilenet_v2.mobilenet(x, num_classes=100)

  # Define a loss function
  loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, 
                                                                      logits = logits))
  # Define an optimizer 
  train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

  # Convert logits to label indexes
  correct_pred = tf.argmax(logits, 1)

  # Define an accuracy metric
  accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

  tf.set_random_seed(1234)
  with tf.Session() as sess:
    
    # Initializing the variables
    sess.run(tf.global_variables_initializer())

    # Training cycle
    for epoch in range(1):
      print('EPOCH', epoch)
      batched_dataset = dataset.batch(BATCH_SIZE)
      iterator = batched_dataset.make_one_shot_iterator()
      next_element = iterator.get_next()
      for i in range(NUM_BATCHES):
        batch = sess.run(next_element)

#         x_train_tensor = tf.convert_to_tensor(batch['image'], dtype=tf.float32)
#         y_train_tensor = tf.convert_to_tensor(batch['label'], dtype=tf.int32)

#         print(x_train_tensor.get_shape())
#         print(y_train_tensor.get_shape())
        
        
        _, accuracy_val = sess.run([train_op, accuracy], feed_dict={x: batch['image'], y: batch['label']})
        tf.print(sess.run(loss))
#         print("Loss: ", loss.eval())
      print('DONE WITH EPOCH')
    

5
Failed to create op 0: Op(op=<function add_arg_scope.<locals>.func_with_args at 0x7f127281d158>, params={'stride': 2, 'num_outputs': 32, 'kernel_size': [3, 3]}, multiplier_func=<function add_arg_scope.<locals>.func_with_args at 0x7f12674c4c80>) params: {'stride': 2, 'num_outputs': 32, 'kernel_size': [3, 3]}


TypeError: ignored